# Load Data 

The main aim of this notebook is to load data into database.  

## Configure the environment 

### Import required libraries

In [1]:
import sqlite3
import pandas as pd

## Connect to Database

In [2]:
database_name = "my_demo_db.db"

con = sqlite3.connect(database_name)

In [3]:
# Utility function
def execute_sql(con: sqlite3.Connection, sql: str):
    """
    Executes the given SQL command using the provided SQLite connection.

    Parameters:
    con (sqlite3.Connection): The SQLite connection object.
    sql (str): The SQL command to be executed.

    Returns:
    sqlite3.Cursor: The cursor object resulting from the executed SQL command.

    Raises:
    Exception: Re-raises any exceptions that occur during the execution of the SQL command.
    """
    try:
        # Use the connection context manager to ensure the command is executed and committed if successful
        with con:
            res = con.execute(sql)  # Execute the SQL command and store the resulting cursor
    except Exception as e:
        # Re-raise any exception that occurs during execution for further handling
        raise e
    
    return res  # Return the cursor resulting from the executed SQL command

## Load data from file 

In [4]:
scale = 100 # The number of production sites as generated by synthetic input generator
          # in step: 05_00_input_data_ generator.ipynb
scale = f"{scale:02}"

### Product Data

In [5]:
# Read from data from file
in_df = pd.read_csv(f'./resources/data/input/product_data_{scale}_site.csv') # input for multisite optimization workflows 
display(in_df.head())
display(in_df.dtypes)

,ProductionSiteID,ProductID,RawMaterialPerProduct,RawMaterialCost,MinimumMarketDemandPerProduct,MaximumMarketDemandPerProduct,ProductPrice
0,S001,P_S001_A_01,13.0,10,1,20,274.0
1,S001,P_S001_A_02,11.0,10,0,20,255.0
2,S001,P_S001_A_03,14.0,10,1,20,282.0
3,S001,P_S001_A_04,12.0,10,1,20,265.0
4,S001,P_S001_A_05,13.0,10,0,20,276.0


ProductionSiteID                  object
ProductID                         object
RawMaterialPerProduct            float64
RawMaterialCost                    int64
MinimumMarketDemandPerProduct      int64
MaximumMarketDemandPerProduct      int64
ProductPrice                     float64
dtype: object

In [6]:
# Create Table 
table_name = 'ProductData'
sql = f"""
CREATE TABLE IF Not Exists {table_name} (
    ProductionSiteID VARCHAR(30),
    ProductID VARCHAR(30),
    RawMaterialPerProduct FLOAT,
    RawMaterialCost FLOAT,
    MinimumMarketDemandPerProduct INTEGER,
    MaximumMarketDemandPerProduct INTEGER,
    ProductPrice FLOAT,
    PRIMARY KEY (ProductionSiteID, ProductID)
);
"""
execute_sql(con=con, sql=sql)

# Load data to DB 
in_df.to_sql(con=con, name=table_name, index=False, if_exists='append')

# check 
sql = f"""SELECT * FROM {table_name}; """
table_df = pd.read_sql(sql=sql, con=con)
display(table_df.head())
display(f"Shape: {table_df.shape}")
display(table_df.dtypes)

,ProductionSiteID,ProductID,RawMaterialPerProduct,RawMaterialCost,MinimumMarketDemandPerProduct,MaximumMarketDemandPerProduct,ProductPrice
0,S001,P_S001_A_01,13.0,10.0,1,20,274.0
1,S001,P_S001_A_02,11.0,10.0,0,20,255.0
2,S001,P_S001_A_03,14.0,10.0,1,20,282.0
3,S001,P_S001_A_04,12.0,10.0,1,20,265.0
4,S001,P_S001_A_05,13.0,10.0,0,20,276.0


'Shape: (3000, 7)'

ProductionSiteID                  object
ProductID                         object
RawMaterialPerProduct            float64
RawMaterialCost                  float64
MinimumMarketDemandPerProduct      int64
MaximumMarketDemandPerProduct      int64
ProductPrice                     float64
dtype: object

### Product Machine Data

In [7]:
# Read from data from file
in_df = pd.read_csv(f'./resources/data/input/product_machine_data_{scale}_site.csv')
display(in_df.head())
display(in_df.dtypes)

,ProductionSiteID,ProductID,MachineID,ProductMachineLaborTimeInMin
0,S001,P_S001_A_01,Machine1,20.0
1,S001,P_S001_A_01,Machine2,16.0
2,S001,P_S001_A_01,Machine3,15.0
3,S001,P_S001_A_02,Machine1,24.0
4,S001,P_S001_A_02,Machine2,18.0


ProductionSiteID                 object
ProductID                        object
MachineID                        object
ProductMachineLaborTimeInMin    float64
dtype: object

In [8]:
# Create Table 
table_name = 'ProductMachineData'
sql = f"""
CREATE TABLE IF Not Exists {table_name} (
    ProductionSiteID VARCHAR(30),
    ProductID VARCHAR(30),
    MachineID VARCHAR(30),
    ProductMachineLaborTimeInMin FLOAT,
    PRIMARY KEY (ProductionSiteID, ProductID, MachineID)
);
"""
execute_sql(con=con, sql=sql)

# Load data to DB 
in_df.to_sql(con=con, name=table_name, index=False, if_exists='append')

# check 
sql = f"""SELECT * FROM {table_name}; """
table_df = pd.read_sql(sql=sql, con=con)
display(table_df.head())
display(f"Shape: {table_df.shape}")
display(table_df.dtypes)

,ProductionSiteID,ProductID,MachineID,ProductMachineLaborTimeInMin
0,S001,P_S001_A_01,Machine1,20.0
1,S001,P_S001_A_01,Machine2,16.0
2,S001,P_S001_A_01,Machine3,15.0
3,S001,P_S001_A_02,Machine1,24.0
4,S001,P_S001_A_02,Machine2,18.0


'Shape: (9000, 4)'

ProductionSiteID                 object
ProductID                        object
MachineID                        object
ProductMachineLaborTimeInMin    float64
dtype: object

### Machine Data

In [9]:
# Read from data from file
in_df = pd.read_csv(f'./resources/data/input/machine_data_{scale}_site.csv')
display(in_df.head())
display(in_df.dtypes)

,ProductionSiteID,MachineID,MachineWeeklyAvailableTimeInHour,MachineCostPerHour
0,S001,Machine1,121.0,25.0
1,S001,Machine2,104.0,32.0
2,S001,Machine3,82.0,45.0
3,S002,Machine1,122.0,24.0
4,S002,Machine2,100.0,31.0


ProductionSiteID                     object
MachineID                            object
MachineWeeklyAvailableTimeInHour    float64
MachineCostPerHour                  float64
dtype: object

In [10]:
# Create Table 
table_name = 'MachineData'
sql = f"""
CREATE TABLE IF Not Exists {table_name} (
    ProductionSiteID VARCHAR(30),
    MachineID VARCHAR(30),
    MachineWeeklyAvailableTimeInHour FLOAT,
    MachineCostPerHour FLOAT,
    PRIMARY KEY (ProductionSiteID, MachineID)
);
"""
execute_sql(con=con, sql=sql)

# Load data to DB 
in_df.to_sql(con=con, name=table_name, index=False, if_exists='append')

# check 
sql = f"""SELECT * FROM {table_name}; """
table_df = pd.read_sql(sql=sql, con=con)
display(table_df.head())
display(f"Shape: {table_df.shape}")
display(table_df.dtypes)

,ProductionSiteID,MachineID,MachineWeeklyAvailableTimeInHour,MachineCostPerHour
0,S001,Machine1,121.0,25.0
1,S001,Machine2,104.0,32.0
2,S001,Machine3,82.0,45.0
3,S002,Machine1,122.0,24.0
4,S002,Machine2,100.0,31.0


'Shape: (300, 4)'

ProductionSiteID                     object
MachineID                            object
MachineWeeklyAvailableTimeInHour    float64
MachineCostPerHour                  float64
dtype: object

### Inventory Data

In [11]:
# Read from data from file
in_df = pd.read_csv(f'./resources/data/input/inventory_data_data_{scale}_site.csv')
display(in_df.head())
display(in_df.dtypes)

,ProductionSiteID,RawMaterialInventory
0,S001,11825.641052
1,S002,11128.419218
2,S003,12296.680554
3,S004,11785.774006
4,S005,11438.821988


ProductionSiteID         object
RawMaterialInventory    float64
dtype: object

In [12]:
# Create Table 
table_name = 'InventoryData'
sql = f"""
CREATE TABLE IF Not Exists {table_name} (
    ProductionSiteID VARCHAR(30),
    RawMaterialInventory FLOAT, 
    PRIMARY KEY (ProductionSiteID)
);
"""
execute_sql(con=con, sql=sql)

# Load data to DB 
in_df.to_sql(con=con, name=table_name, index=False, if_exists='append')

# check 
sql = f"""SELECT * FROM {table_name}; """
table_df = pd.read_sql(sql=sql, con=con)
display(table_df.head())
display(f"Shape: {table_df.shape}")
display(table_df.dtypes)

,ProductionSiteID,RawMaterialInventory
0,S001,11825.641052
1,S002,11128.419218
2,S003,12296.680554
3,S004,11785.774006
4,S005,11438.821988


'Shape: (100, 2)'

ProductionSiteID         object
RawMaterialInventory    float64
dtype: object

## Close Connection 

In [13]:
con.close()

# END